In [1]:
# import pyspark
from pyspark import SparkContext, SparkConf
cf = SparkConf()
cf.set("spark.submit.deployMode","client")
sc = SparkContext.getOrCreate(cf)
from pyspark.sql import SparkSession
spark = SparkSession \
	    .builder \
	    .appName("Python Spark SQL basic example") \
	    .config("spark.some.config.option", "some-value") \
	    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/11 23:02:50 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
23/12/11 23:02:50 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
23/12/11 23:02:50 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
23/12/11 23:02:50 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator


In [2]:
# import processed data and select the correspooind coloumns
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("ParquetRead").getOrCreate()
custom_schema = "domain STRING, product STRING, label DOUBLE"
ads_df = spark.read.schema(custom_schema).parquet("/shared/CS-GY-6513/projects/WildLife/processed-data-oct")
ads_df.createOrReplaceTempView("ads_view")
ads_df.show(20, truncate=False)

+------------+-----------------------------------------------------------------------------------------------------+-----+
|domain      |product                                                                                              |label|
+------------+-----------------------------------------------------------------------------------------------------+-----+
|picclick.com|☆ HUGE LIST Vintage & New Individual Kinder Surprise Toys - Pick Your Own! ☆ $1.45 - PicClick        |0.0  |
|picclick.com|ðŞ"¥ATLANTIS CHRONICLES 4 CGC 9.8 White Pages DC Comics 1990 Aquaman II 1st Kordax $128.00 - PicClick|0.0  |
|picclick.com|1/2 OZ .999 Silver proof 2016 Mexico Libertad .999 $34.99 - PicClick                                 |0.0  |
|picclick.com|1 REGAL OSTRICH Martha Stewart Copper Cookie Cutter Noah's Ark Set 2 $5.99 - PicClick                |0.0  |
|picclick.com|10" A PAIR China Porcelain Ming dynasty xuande Underglaze red fish algae Jar pot $1,118.00 - PicClick|0.0  |
|picclick.com|10

In [3]:
ads_df.count()

375726

In [4]:
# filtering out the duplicates
ads_df2 = ads_df.select('product').distinct().count()
print(ads_df2)

182537


In [5]:
product_distinct = spark.sql("SELECT DISTINCT product, label FROM ads_view")
product_distinct.show()
product_distinct.count()

+--------------------+-----+
|             product|label|
+--------------------+-----+
|METAL 24" UNITED ...|  0.0|
|TIGNANELLO PEBBLE...|  0.0|
|Susan B Anthony (...|  0.0|
|Wyoming, Patches,...|  1.0|
|Postcards, United...|  0.0|
|National Guitar F...|  0.0|
|Victoria Secret O...|  0.0|
|African Ostrich (...|  0.0|
|Squishmallows 18"...|  0.0|
|Ground beetle Car...|  1.0|
|Continental Postc...|  1.0|
|Wall Mounted Stor...|  1.0|
|Halibut in Clothi...|  1.0|
|Nile crocodile | ...|  1.0|
|Cast Iron Enamel ...|  0.0|
|Jaguar Cd Changer...|  0.0|
|Lagos Jewelry FOR...|  0.0|
|Hoodies & Sweatsh...|  1.0|
|revell hot-hatch ...|  0.0|
|JJM Canidae Anima...|  0.0|
+--------------------+-----+
only showing top 20 rows



182537

In [6]:
# look for animal products
animal_product_df = product_distinct.filter(product_distinct.label == 1.0)
animal_product_df.show()
animal_product_df.count()

+--------------------+-----+
|             product|label|
+--------------------+-----+
|FOSSILIZED 1.07 I...|  1.0|
|FULL HEAD 100% Na...|  1.0|
|Wheel Hub Caps & ...|  1.0|
|Ostrich in Fast F...|  1.0|
|Nile crocodile | ...|  1.0|
|ARIAT MENS CROSS ...|  1.0|
|Bouchon de fil fl...|  1.0|
|GREAT WHITE Shark...|  1.0|
|Real Fossil Sand ...|  1.0|
|Wolverine Skull N...|  1.0|
|The Taxidermy Sto...|  1.0|
|'Wolf' for sale i...|  1.0|
|Ground beetle for...|  1.0|
| Knoblauchhacker ...|  1.0|
|Lace Fishnet Herr...|  1.0|
|Virginia sabbia f...|  1.0|
|Glacier Wear - Fu...|  1.0|
|Canvasback Duck S...|  1.0|
|Marine Rope, Boat...|  1.0|
|Halibut in Clothi...|  1.0|
+--------------------+-----+
only showing top 20 rows



35792

In [7]:
# Not animal products
Un_animal_product_df = product_distinct.filter(product_distinct.label == 0.0)
Un_animal_product_df.show()
Un_animal_product_df.count()

+--------------------+-----+
|             product|label|
+--------------------+-----+
|METAL 24" UNITED ...|  0.0|
|TIGNANELLO PEBBLE...|  0.0|
|Susan B Anthony (...|  0.0|
|Postcards, United...|  0.0|
|National Guitar F...|  0.0|
|Victoria Secret O...|  0.0|
|African Ostrich (...|  0.0|
|Squishmallows 18"...|  0.0|
|Cast Iron Enamel ...|  0.0|
|Jaguar Cd Changer...|  0.0|
|Lagos Jewelry FOR...|  0.0|
|revell hot-hatch ...|  0.0|
|JJM Canidae Anima...|  0.0|
|Carcharhinus falc...|  0.0|
|Pirate Wig FOR SA...|  0.0|
|Volkswagen Beetle...|  0.0|
|Nile Crocodile Cr...|  0.0|
|Infinity Studio M...|  0.0|
|Dir En Grey, Arti...|  0.0|
|Furniture Accesso...|  0.0|
+--------------------+-----+
only showing top 20 rows



146745

In [8]:
# import the threatened species data from appendix I
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("MyApp") \
    .getOrCreate()

endangered_species_df = spark.read.csv('/user/yz10589_nyu_edu/Species1.csv', header=True, inferSchema=True)
endangered_species_df.show()
endangered_species_df.count()

+--------------------+
|             Species|
+--------------------+
|Antilocapra ameri...|
| Addax nasomaculatus|
|          Bos gaurus|
|           Bos mutus|
|         Bos sauveli|
|Bubalus depressic...|
| Bubalus mindorensis|
|    Bubalus quarlesi|
|     Capra falconeri|
|Capricornis milne...|
| Capricornis rubidus|
|Capricornis sumat...|
|    Capricornis thar|
|Cephalophus jentinki|
|     Gazella cuvieri|
|  Gazella leptoceros|
|Hippotragus niger...|
| Naemorhedus baileyi|
|Naemorhedus caudatus|
|   Naemorhedus goral|
+--------------------+
only showing top 20 rows



632

In [9]:
# define the function to do the comparison that check if the ads contain threatened species information
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import BooleanType

values_to_check = endangered_species_df.select("Species").rdd.flatMap(lambda x: x).collect()

def value_is_contained(val):
    return any(x in val for x in values_to_check)

value_is_contained_udf = udf(value_is_contained, BooleanType())

result_df = animal_product_df.filter(value_is_contained_udf(animal_product_df.product))

result_df.show()
result_df.count()

+--------------------+-----+
|             product|label|
+--------------------+-----+
|Canis lupus in Se...|  1.0|
|Congo #246 EPL Pr...|  1.0|
|Crocodile skull C...|  1.0|
|Bra & Panty Set L...|  1.0|
|VERKAUF 251 - Nil...|  1.0|
|26mm XL Panatime ...|  1.0|
|1 of 1 Crocodile ...|  1.0|
|Men's Crossdresse...|  1.0|
|Damen Sexy Shiny ...|  1.0|
|Canis lupus en ve...|  1.0|
|26mm XL Panatime ...|  1.0|
|Crâne crocodile C...|  1.0|
|Pantaloni da pass...|  1.0|
|Kuscheltiere,Echt...|  1.0|
|26mm Panatime Bro...|  1.0|
|Sheln Heels Gold ...|  1.0|
|2023 Brand New Pa...|  1.0|
|ППлПсатая Ќарка О...|  1.0|
|Canis lupus in Gr...|  1.0|
|Lace Fishnet Herr...|  1.0|
+--------------------+-----+
only showing top 20 rows



417

In [10]:
# calculation of percentage
417/35792

0.011650648189539562